In [4]:
import requests
import pandas as pd
from functools import reduce

pd.options.display.float_format = '{:.0f}'.format


In [11]:
# Last n runs to review
params = {"n": 10} # you can adjust this, I wouldn't do any more than 20 though

# Get run parameter data for last n runs
url = "https://xraiapi-ba66c372be3f.herokuapp.com/database/last_n_runs/tbl_model_run_params"
model_param_data_json = requests.get(url, params=params)
model_param_data = model_param_data_json.json() 
model_param_data = pd.DataFrame(model_param_data)
model_param_data = model_param_data[["cflt_run_id", "cint_nr_agents"]].drop_duplicates()


# Get local state data for last n runs
url = "https://xraiapi-ba66c372be3f.herokuapp.com/database/last_n_runs/tbl_local_state"
local_data_json = requests.get(url, params=params)
local_data = local_data_json.json() 
local_data = pd.DataFrame(local_data)

# Get stop information for each run to know how many episodes we should expect to be recorded
url = "https://xraiapi-ba66c372be3f.herokuapp.com/database/last_n_runs/tbl_model_runs"
run_data_json = requests.get(url, params=params)
run_data = run_data_json.json() 
run_data = pd.DataFrame(run_data)
run_data_pauses = run_data[run_data["cstr_model_status"]=="paused"][["cflt_run_id","cstr_model_status"]]
run_data_pauses = run_data_pauses.groupby("cflt_run_id").agg(pause_count=("cstr_model_status", "count")).reset_index()


In [12]:
# Get the total number of episodes recorded and the max episode count. 
# If the number of episodes recroded is less than the episode count, that indicates that model output wasn't properly committed.
# NOTE: When everything is working as expected, the episode_count value will always be greater than max_episode since the starting episode is 0. 
episode_counts = local_data.groupby('cflt_run_id').agg(
    episode_count=('cint_episode_id', 'nunique'),  
    max_episode=('cint_episode_id', 'max')       
).reset_index()
# NOTE: When everything is working as expected, the episode_count value will always be greater than max_episode since the starting episode is 0. 
# To check for inconsistencies, 1 is substractdd from episode_count to address this. 
episode_counts["Episode Inconsistencies"] = episode_counts["max_episode"] - (episode_counts["episode_count"]-1) 

In [13]:
to_join = [model_param_data, run_data_pauses, episode_counts]
review_df = reduce(lambda left, right: pd.merge(left, right, on='cflt_run_id', how='left'), to_join)
review_df.columns = ["Run ID", "# Drones", "# Pauses", "# Episodes Recorded", "Max Episode Recorded","Episode Inconsistencies"]
display(review_df.sort_values("Run ID"))

,Run ID,# Drones,# Pauses,# Episodes Recorded,Max Episode Recorded,Episode Inconsistencies
0,1733091454717,3,NaN,70,69,0
1,1733093967272,20,NaN,490,489,0
2,1733095209977,20,NaN,10,9,0
3,1733095700548,20,NaN,280,279,0
4,1733095982742,20,1,30,29,0
5,1733097772266,20,NaN,480,479,0
6,1733097996639,5,NaN,490,499,10
7,1733098356452,5,NaN,500,499,0
8,1733098468376,5,NaN,500,499,0
9,1733100355965,20,NaN,490,489,0
